In [ ]:
import os
os.chdir('/scratch2/jhsiao/projects/notebooks/book-top-directory/metazoa16-mapped-repaired/1-mapping-pacbio')

In [ ]:
%%bash
#001-wget-genome-pacbio-transcriptome

mkdir output || echo ""
mkdir output/source-data/ || echo ""

# PacBio transcriptome NCBI TSA
wget http://igc2.salk.edu/illumina/runs/PacBio_200525/Chalasani_Eric_200525/EE04/ISO-seq3/hq_transcripts.fasta -P output/

# Ensembl genome mRNA
wget http://ftp.ensemblgenomes.org/pub/metazoa/release-52/fasta/mnemiopsis_leidyi/cdna/Mnemiopsis_leidyi.MneLei_Aug2011.cdna.all.fa.gz -P output/source-data/

# Ensembl genome CDS
wget http://ftp.ensemblgenomes.org/pub/metazoa/release-52/fasta/mnemiopsis_leidyi/cds/Mnemiopsis_leidyi.MneLei_Aug2011.cds.all.fa.gz -P output/source-data/

# Ensembl genome AA
wget http://ftp.ensemblgenomes.org/pub/metazoa/release-52/fasta/mnemiopsis_leidyi/pep/Mnemiopsis_leidyi.MneLei_Aug2011.pep.all.fa.gz -P output/source-data/

# Ensembl genome GFF
wget http://ftp.ensemblgenomes.org/pub/metazoa/release-52/gff3/mnemiopsis_leidyi/Mnemiopsis_leidyi.MneLei_Aug2011.52.gff3.gz -P output/source-data/ 

gunzip output/source-data/*


In [ ]:
%%bash
#002-evigene


mkdir output/2-evigene || echo ""
cd output/2-evigene
cp ../source-data/hq_transcripts.fasta . &&

trformat.pl  -output hq_transcripts-evigene_headers.fasta  -input hq_transcripts.fasta -format=pacbio &&
tr2aacds4.pl -mrnaseq hq_transcripts-evigene_headers.fasta -NCPU=40 -MAXMEM=900000 -MINAA=30 -logfile 2-log-hq_transcripts -tidyup 



In [ ]:
#003-python-parse-evigene-T1s


# >NonamEVm000019t1 type=mRNA; aalen=3767,77%,complete; clen=14673; offs=2676-13979; organism=No_name; evgclass=main; oid=evgpacbUnnamedSample_HQ_transcript/1;
# >NonamEVm000019t1 type=CDS; aalen=3767,77%,complete; clen=14673; offs=2676-13979; organism=No_name; evgclass=main; oid=evgpacbUnnamedSample_HQ_transcript/1; codepot=Code/0.53;
# >NonamEVm000019t1 type=protein; aalen=3767,77%,complete; clen=14673; offs=2676-13979; organism=No_name; evgclass=main; oid=evgpacbUnnamedSample_HQ_transcript/1; codepot=Code/0.53;

output_map = open( 'output/3-map-pacbio-to-evigeneT1-headers', 'w' )

file_types = [ 'mrna', 'cds', 'aa' ]

for next_type in file_types:

    file_in = 'output/2-evigene/okayset/hq_transcripts-evigene_headers.okay.' + next_type
    input_fasta = open( file_in, 'r' )
    
    file_out = 'output/3-Mnemiopsis-leidyi-pacbio-evigeneT1.' + next_type
    output_fasta = open( file_out, 'w' )

    header_seq = {}
    source_ids = []
    for next_line in input_fasta:

        if next_line[ 0 ] == '>':
            info_1 = next_line[ 1:-1 ].split( ' ' )
            info_2 = info_1[ 0 ].split( 't' )

            if info_2[ -1 ] == '1':

                new_id = 'Mnemiopsis-leidyi-' + info_1[ 0 ]

                info_3 = next_line[ :-1 ].split( ' oid=' )
                info_4 = info_3[ -1 ].split( ';' )
                info_5 = info_4[ 0 ].split( '_' )
                source_id = '_'.join( info_5[ 1: ] )
                                
                # this will drop additional utr isoforms of a sequence - evigene adds them as additional T1s
                # but I think this is incorrect
                # from the perspective of T1s should represent a single isoform of a gene in the genome
                
                if new_id in header_seq.keys():
                    keeper = False

                else:

                    source_id_clean_list = source_id.split( 'utrorf' )
                    source_id_clean = source_id_clean_list[ 0 ]
                    
                    if len( source_id_clean_list ) > 1:
                        utr_issue = 'utrorf;' + info_4[ 1 ].split( ' ' )[ 0 ]

                    else:
                        utr_issue = ''
                        
                    header_seq[ new_id ] = ''
                    keeper = True

                    output = next_type + '\t' + new_id + '\t' + source_id_clean + '\t' + utr_issue +  '\n'
                    output_map.write( output )
           
            else:
                keeper = False

        else:

            if keeper == True:
                header_seq[ new_id ] = header_seq[ new_id ] + next_line[ :-1 ]

            else:
                pass

    for next_id in header_seq.keys():

        header = '>' + next_id 
        sequence = header_seq[ next_id ]
        output = header + '\n' + sequence + '\n'
        output_fasta.write( output )

    input_fasta.close()
    output_fasta.close()

output_map.close()



In [ ]:
%%bash
#004-mkblastdb-genome


/usr/bin/makeblastdb -in output/Mnemiopsis_leidyi.MneLei_Aug2011.cdna.all.fa -parse_seqids -dbtype nucl
/usr/bin/makeblastdb -in output/Mnemiopsis_leidyi.MneLei_Aug2011.cds.all.fa -parse_seqids -dbtype nucl
/usr/bin/makeblastdb -in output/Mnemiopsis_leidyi.MneLei_Aug2011.pep.all.fa -parse_seqids -dbtype prot



In [ ]:
%%bash
#005-blastn-pacbio_X_genome

/usr/bin/blastn -db output/Mnemiopsis_leidyi.MneLei_Aug2011.cds.all.fa -outfmt 6 -out output/5-blast-report-pacbio_X_genome-Mnemiopsis-leidyi-CDS -max_hsps 1 -query output/3-Mnemiopsis-leidyi-pacbio-evigeneT1.mrna -evalue 1e-3 -num_threads 40



In [ ]:
#006-python-parse-report-and-update-gene-models-MRNA

# UnnamedSample_HQ_transcript/0   ML16643a-RA     99.804  5090    10      0       9617    14706   1048    6137    0.0     9345

input_report = open( 'output/5-blast-report-pacbio_X_genome-Mnemiopsis-leidyi-CDS', 'r' )
input_transcriptome = open( 'output/3-Mnemiopsis-leidyi-pacbio-evigeneT1.mrna', 'r' )
input_genome = open( 'output/Mnemiopsis_leidyi.MneLei_Aug2011.cdna.all.fa', 'r' )

output_fasta = open( 'output/6-Mnemiopsis_leidyi-genome-pacbio-updates.mrna', 'w' )
output_map = open( 'output/6-map-sourceid-to-updateid', 'w' )
output_difference = open( 'output/6-pacbio-minus-gene-lengths', 'w' )
output_stats = open( 'output/6-stats', 'w' )
output_report_stats = open( 'output/6-stats-more-than-one-gene-at-0.0', 'w' )

# read in genome sequences
genome_seq = {}
transcriptome_seq = {}
genome_original_letters = ''
for next_line in input_genome:

    if next_line[ 0 ] == '>':
        header_id = next_line[ 1:-1 ].split( ' ' )[ 0 ]
        genome_seq[ header_id ] = ''

    else:
        sequence = next_line[ :-1 ]
        genome_seq[ header_id ] = genome_seq[ header_id ] + sequence
        genome_original_letters = genome_original_letters +  sequence
        
# read in pacbio transcriptome sequences
for next_line in input_transcriptome:

    if next_line[ 0 ] == '>':
        header = next_line[ 1:-1 ]
        transcriptome_seq[ header ] = ''

    else:
        transcriptome_seq[ header ] = transcriptome_seq[ header ] + next_line[ :-1 ]

# 1.  qseqid      query or source (e.g., gene) sequence id
# 2.  sseqid      subject  or target (e.g., reference genome) sequence id
# 3.  pident      percentage of identical matches
# 4.  length      alignment length (sequence overlap)
# 5.  mismatch    number of mismatches
# 6.  gapopen     number of gap openings
# 7.  qstart      start of alignment in query
# 8.  qend        end of alignment in query
# 9.  sstart      start of alignment in subject
# 10.  send        end of alignment in subject
# 11.  evalue      expect value
# 12.  bitscore    bit score

# read in blast report
transcript_genes = {}
gene_transcripts = {}
transcript_report = {}
for next_line in input_report:

    info = next_line.split( '\t' )
    transcript_id = info[ 0 ]
    gene_id = info[ 1 ]
    evalue = info[ 10 ]
    percent_id = info[ 2 ]
    align_length = info[ 3 ]
    mismatch = info[ 4 ]
    bitscore = info[ 11 ]

    transcript_stats = [ transcript_id, gene_id, evalue, percent_id, align_length, mismatch, bitscore ]
    
    if gene_id in genome_seq.keys():
        
        if evalue == '0.0':
 
            # add transcript and genes to transcript_genes dictionary 
            if transcript_id in transcript_genes.keys():
                transcript_genes[ transcript_id ].append( gene_id )
                transcript_report[ transcript_id ][ gene_id ] = transcript_stats

            else:
                transcript_genes[ transcript_id ] = []
                transcript_genes[ transcript_id ].append( gene_id )

                transcript_report[ transcript_id ] = {}
                transcript_report[ transcript_id ][ gene_id ] = transcript_stats
                
            # add gene and transcript to gene_transcripts dictionary
            if gene_id in gene_transcripts.keys():
                gene_transcripts[ gene_id ].append( transcript_id )

            else:
                gene_transcripts[ gene_id ] = []
                gene_transcripts[ gene_id ].append( transcript_id )
                                            
        else:
            pass
    else:
        pass

# parse for genes with just one hit have evalue 0.0
gene_replacement = {}
for next_transcript in transcript_genes.keys():

    if len( transcript_genes[ next_transcript ] ) > 1:
            
        for next_gene in transcript_genes[ next_transcript ]:
            gene_report_stats = transcript_report[ next_transcript ][ next_gene ]
            output = ''
            for next_stat in gene_report_stats:
                output = output + next_stat + '\t'
            output = output[ :-1 ] + '\n'
            output_report_stats.write( output )

    else:
        gene_id = transcript_genes[ next_transcript ][ 0 ]
        gene_sequence = genome_seq[ gene_id ]
        transcript_sequence = transcriptome_seq[ next_transcript ]

        gene_base = gene_id.split( '-' )[ 0 ]

        if gene_base in gene_replacement.keys():
            if len( transcriptome_seq[ next_transcript ] ) > len( gene_replacement[ gene_base ] ):
                gene_replacement[ gene_base ] = next_transcript
            else:
                pass
        else:
            gene_replacement[ gene_base ] = next_transcript

# update genome gene model if there is a  pacbio sequence - assume pacbio sequence is better since read is real - gene model is hypothesized based on evidence of varying unknown specific quality
# >ML49653a-RA cdna supercontig:MneLei_Aug2011:ML4965:35353:38358:-1 gene:ML49653a gene_biotype:protein_coding transcript_biotype:protein_coding_pacbioMnemiopsis-leidyi-NonamEVm004368t1

genome_updated_letters = ''
for next_gene in genome_seq.keys():
    next_gene_base = next_gene.split( '-' )[ 0 ]
    
    if next_gene_base in gene_replacement.keys():
        transcript_id = gene_replacement[ next_gene_base ]
        next_gene_info = next_gene.split( ' ' )
        transcript_info_1 = transcript_id.split( ' ' )
        transcript_info_2 = transcript_info_1[ 0 ].split( '-' )
        gene_id = next_gene_info[ 0 ]
        original_seq = genome_seq[ gene_id ]
        updated_gene_id = gene_id + '___pacbio' + transcript_info_2[ -1 ] #+ ' ' + ' '.join( next_gene_info[ 1: ] )
        updated_seq = transcriptome_seq[ transcript_id ]

        difference = str( len( updated_seq ) - len( original_seq ) )
        output = 'PacBio ' + transcript_id + ' ' + str( len( updated_seq ) ) + '\t Gene ' + gene_id + ' ' + str( len( original_seq ) ) + '\t PacBio - Gene = ' + difference + ' nts mRNA\n'
        output_stats.write( output )
        output = str( len( updated_seq ) ) + '\t' + str( len( original_seq ) ) + '\t' + difference + '\n'
        output_difference.write( output)

        output = '>' + updated_gene_id + '\n' + updated_seq + '\n'
        output_fasta.write( output )
        genome_updated_letters = genome_updated_letters + updated_seq

        output = next_gene + '\t' + transcript_id + '\t' + updated_gene_id + '\n'
        output_map.write( output )
        
    else:
        sequence = genome_seq[ next_gene ]
        output = '>' + next_gene + '\n' + sequence + '\n'
        output_fasta.write( output )
        genome_updated_letters = genome_updated_letters + sequence

output = 'Original gene models = ' + str( len( genome_original_letters ) ) + ' nucleotide mRNA sequence'
print( output)
output_stats.write( output )
output = 'Updated gene models = ' + str( len( genome_updated_letters ) ) + ' nucleotide mRNA sequence'
print( output)
output_stats.write( output )
output = 'Number of nucleotides added to mRNA gene models = ' + str( len( genome_updated_letters ) - len( genome_original_letters ) )
print( output )
output_stats.write( output )

input_report.close()
input_transcriptome.close()
input_genome.close()
output_fasta.close()
output_map.close()
output_difference.close()
output_stats.close()
output_report_stats.close()



In [ ]:
#007-python-parse-report-and-update-gene-models-CDS

# UnnamedSample_HQ_transcript/0   ML16643a-RA     99.804  5090    10      0       9617    14706   1048    6137    0.0     9345

input_report = open( 'output/5-blast-report-pacbio_X_genome-Mnemiopsis-leidyi-CDS', 'r' )
input_transcriptome = open( 'output/3-Mnemiopsis-leidyi-pacbio-evigeneT1.cds', 'r' )
input_genome = open( 'output/Mnemiopsis_leidyi.MneLei_Aug2011.cds.all.fa', 'r' )

output_fasta = open( 'output/7-Mnemiopsis_leidyi-genome-pacbio-updates.cds', 'w' )
output_map = open( 'output/7-map-sourceid-to-updateid', 'w' )
output_difference = open( 'output/7-pacbio-minus-gene-lengths', 'w' )
output_stats = open( 'output/7-stats', 'w' )
output_report_stats = open( 'output/7-stats-more-than-one-gene-at-0.0', 'w' )

# read in genome sequences
genome_seq = {}
transcriptome_seq = {}
genome_original_letters = ''
for next_line in input_genome:

    if next_line[ 0 ] == '>':
        header_id = next_line[ 1:-1 ].split( ' ' )[ 0 ]
        genome_seq[ header_id ] = ''

    else:
        sequence = next_line[ :-1 ]
        genome_seq[ header_id ] = genome_seq[ header_id ] + sequence
        genome_original_letters = genome_original_letters +  sequence
        
# read in pacbio transcriptome sequences
for next_line in input_transcriptome:

    if next_line[ 0 ] == '>':
        header = next_line[ 1:-1 ]
        transcriptome_seq[ header ] = ''

    else:
        transcriptome_seq[ header ] = transcriptome_seq[ header ] + next_line[ :-1 ]

# 1.  qseqid      query or source (e.g., gene) sequence id
# 2.  sseqid      subject  or target (e.g., reference genome) sequence id
# 3.  pident      percentage of identical matches
# 4.  length      alignment length (sequence overlap)
# 5.  mismatch    number of mismatches
# 6.  gapopen     number of gap openings
# 7.  qstart      start of alignment in query
# 8.  qend        end of alignment in query
# 9.  sstart      start of alignment in subject
# 10.  send        end of alignment in subject
# 11.  evalue      expect value
# 12.  bitscore    bit score

# read in blast report
transcript_genes = {}
gene_transcripts = {}
transcript_report = {}
for next_line in input_report:

    info = next_line.split( '\t' )
    transcript_id = info[ 0 ]
    gene_id = info[ 1 ]
    evalue = info[ 10 ]
    percent_id = info[ 2 ]
    align_length = info[ 3 ]
    mismatch = info[ 4 ]
    bitscore = info[ 11 ]

    transcript_stats = [ transcript_id, gene_id, evalue, percent_id, align_length, mismatch, bitscore ]
    
    if gene_id in genome_seq.keys():
        
        if evalue == '0.0':
 
            # add transcript and genes to transcript_genes dictionary 
            if transcript_id in transcript_genes.keys():
                transcript_genes[ transcript_id ].append( gene_id )
                transcript_report[ transcript_id ][ gene_id ] = transcript_stats

            else:
                transcript_genes[ transcript_id ] = []
                transcript_genes[ transcript_id ].append( gene_id )

                transcript_report[ transcript_id ] = {}
                transcript_report[ transcript_id ][ gene_id ] = transcript_stats
                
            # add gene and transcript to gene_transcripts dictionary
            if gene_id in gene_transcripts.keys():
                gene_transcripts[ gene_id ].append( transcript_id )

            else:
                gene_transcripts[ gene_id ] = []
                gene_transcripts[ gene_id ].append( transcript_id )
                                            
        else:
            pass
    else:
        pass

# parse for genes with just one hit have evalue 0.0
gene_replacement = {}
for next_transcript in transcript_genes.keys():

    if len( transcript_genes[ next_transcript ] ) > 1:
            
        for next_gene in transcript_genes[ next_transcript ]:
            gene_report_stats = transcript_report[ next_transcript ][ next_gene ]
            output = ''
            for next_stat in gene_report_stats:
                output = output + next_stat + '\t'
            output = output[ :-1 ] + '\n'
            output_report_stats.write( output )

    else:
        gene_id = transcript_genes[ next_transcript ][ 0 ]
        gene_sequence = genome_seq[ gene_id ]
        transcript_sequence = transcriptome_seq[ next_transcript ]

        gene_base = gene_id.split( '-' )[ 0 ]

        if gene_base in gene_replacement.keys():
            if len( transcriptome_seq[ next_transcript ] ) > len( gene_replacement[ gene_base ] ):
                gene_replacement[ gene_base ] = next_transcript
            else:
                pass
        else:
            gene_replacement[ gene_base ] = next_transcript

# update genome gene model if there is a  pacbio sequence - assume pacbio sequence is better since read is real - gene model is hypothesized based on evidence of varying unknown specific quality
# >ML49653a-RA cdna supercontig:MneLei_Aug2011:ML4965:35353:38358:-1 gene:ML49653a gene_biotype:protein_coding transcript_biotype:protein_coding_pacbioMnemiopsis-leidyi-NonamEVm004368t1

genome_updated_letters = ''
for next_gene in genome_seq.keys():
    next_gene_base = next_gene.split( '-' )[ 0 ]
    
    if next_gene_base in gene_replacement.keys():
        transcript_id = gene_replacement[ next_gene_base ]
        next_gene_info = next_gene.split( ' ' )
        transcript_info_1 = transcript_id.split( ' ' )
        transcript_info_2 = transcript_info_1[ 0 ].split( '-' )
        gene_id = next_gene_info[ 0 ]
        original_seq = genome_seq[ gene_id ]
        updated_gene_id = gene_id + '___pacbio' + transcript_info_2[ -1 ] #+ ' ' + ' '.join( next_gene_info[ 1: ] )
        updated_seq = transcriptome_seq[ transcript_id ]

        difference = str( len( updated_seq ) - len( original_seq ) )
        output = 'PacBio ' + transcript_id + ' ' + str( len( updated_seq ) ) + '\t Gene ' + gene_id + ' ' + str( len( original_seq ) ) + '\t PacBio - Gene = ' + difference + ' nts CDS\n'
        output_stats.write( output )
        output = str( len( updated_seq ) ) + '\t' + str( len( original_seq ) ) + '\t' + difference + '\n'
        output_difference.write( output)

        output = '>' + updated_gene_id + '\n' + updated_seq + '\n'
        output_fasta.write( output )
        genome_updated_letters = genome_updated_letters + updated_seq

        output = next_gene + '\t' + transcript_id + '\t' + updated_gene_id + '\n'
        output_map.write( output )
        
    else:
        sequence = genome_seq[ next_gene ]
        output = '>' + next_gene + '\n' + sequence + '\n'
        output_fasta.write( output )
        genome_updated_letters = genome_updated_letters + sequence

output = 'Original gene models = ' + str( len( genome_original_letters ) ) + ' nucleotide CDS sequence'
print( output)
output_stats.write( output )
output = 'Updated gene models = ' + str( len( genome_updated_letters ) ) + ' nucleotide CDS sequence'
print( output)
output_stats.write( output )
output = 'Number of nucleotides added to CDS gene models = ' + str( len( genome_updated_letters ) - len( genome_original_letters ) )
print( output )
output_stats.write( output )

input_report.close()
input_transcriptome.close()
input_genome.close()
output_fasta.close()
output_map.close()
output_difference.close()
output_stats.close()
output_report_stats.close()



In [ ]:
%%bash
#008-python-update-gene-models-AA


input_cds = open( 'output/7-Mnemiopsis_leidyi-genome-pacbio-updates.cds', 'r' )
input_aa = open( 'output/3-Mnemiopsis-leidyi-pacbio-evigeneT1.aa', 'r' )
input_genome_aa = open( 'output/Mnemiopsis_leidyi.MneLei_Aug2011.pep.all.fa', 'r' )
output_aa = open( 'output/8-Mnemiopsis_leidyi-genome-pacbio-updates.aa', 'w' )

aa_seq = {}
# >Mnemiopsis-leidyi-NonamEVm000019t1
for next_line in input_aa:

    if next_line[ 0 ] == '>':

        evigene_id = next_line[ 1:-1 ].split( '-' )[ -1 ]
        aa_seq[ evigene_id ] = ''

    else:

        sequence = next_line[ :-1 ]
        aa_seq[ evigene_id ] = aa_seq[ evigene_id ] + sequence

cds_seq = {}
# >ML00011a-RA___pacbioNonamEVm009938t1
for next_line in input_cds:

    if next_line[ 0 ] == '>':

        identifier = next_line[ 1:-1 ]
        cds_seq[ identifier ] = ''

    else:

        sequence = next_line[ :-1 ]
        cds_seq[ identifier ] = cds_seq[ identifier ] + sequence

genome_aa_seq = {}
for next_line in input_genome_aa:

    if next_line[ 0 ] == '>':

        ensembl_id = next_line[ 1:-1 ].split( ' ' )[ 0 ]
        genome_aa_seq[ ensembl_id ] = ''

    else:

        sequence = next_line[ :-1 ]
        genome_aa_seq[ ensembl_id ] = genome_aa_seq[ ensembl_id ] + sequence
        
for next_id in cds_seq.keys():

    info = next_id.split( '___' )

    if len( info ) > 1:
        evigene_id = info[ -1 ].split( 'bio' )[ -1 ]
        sequence = aa_seq[ evigene_id ]
        ensembl_id = info[ 0 ].split( '-' )[ 0 ] 

        output = '>' + ensembl_id  + '-PA___pacbio' + evigene_id + '\n' + sequence + '\n'
        output_aa.write( output )

    else:
        info_id = next_id.split( '-' )
        if len( info_id ) > 1:
            ensembl_id = info_id[ 0 ] + '-PA'
        else:
            ensembl_id = info_id[ 0 ]

        sequence = genome_aa_seq[ ensembl_id ]
        output = '>' + ensembl_id + '\n' + sequence + '\n'
        output_aa.write( output )
            
input_cds.close()
input_aa.close()
output_aa.close()

